# Пример для веба

Сделал класс-обертку для DL-модели
умеет делать предсказание класса на картинке, ищет GPU, если его нет использует CPU.


Прошу веб специалистов нашей команды посмотреть данный класс в файле "AgroCode pre DLModelImageClassifier.ipynb".
(если вам удобнее просто *py файл то посмотрите "сейчас я его сделаю")


классу  DLModelImageClassifier будут нужны два файла
path_model  - путь до сохраненной нейросети (архитектура и веса одним файлом) файл "ResNet34_DGL_AdamW_aug_oversampling_batchsize_16_full_model.pth"
path_label_enc - путь до сохраненного кодировщика имен классов файл "label_encoder.pkl"


Оба файла в папке "model"
И попробовать сделать тестовый веб-сервер, с сайтом. Чтобы на сайт можно было загрузить одну картинку.
(Картинки есть в папке "pics_example"). И Чтобы сервер напечатал ответ от нейросети.

Сбрасываем ноутбук и запускаем его начиная отсюда, чтобы ничего из предидущего в памяти ВМ не было.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import os

import torch
import pickle
import numpy as np
import pandas as pd
 
from PIL import Image
 
from torchvision import transforms, models
from sklearn.preprocessing import LabelEncoder

In [ ]:
class DLModelImageClassifier(object):
    """Класс-обертка над моделью нейросети для классифкации картинок"""

    def __init__(self, path_model, path_label_enc):
        """
        Parameters:

        path_model : str, путь до сохраненной нейросети (архитектура и веса модели PyTorch файлом)
        path_label_enc : str, путь до сохраненного кодировщика имен классов (sklearn.preprocessing.LabelEncoder)
        """

        # загружаем модель
        # на всякий случай грузим модель для cpu
        self.model = torch.load(path_model, map_location=torch.device('cpu'))
        self.model.eval()

        # загружаем кодировщик названий классов
        with open(path_label_enc, 'rb') as f:
            label_enc = pickle.load(f)

        self.label_enc = label_enc

        # определим трансформации картинок для предобработки
        self.our_transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        # Определим устройство, если есть GPU используем его
        # если GPU нет, то CPU
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # перенесем DL-модель на устройство
        self.model.to(self.device)

        # для предсказания на папке файлов
        self.prediction_images_values = []
        self.prediction_images_names = []

    def transform_image(self, image_path):
        """делает необходимую трансформацию картинки, расположенной по пути image_path
         для модели нейросети для классифкации картинок"""
        image = Image.open(image_path)

        return self.our_transforms(image).unsqueeze(0)

    def get_prediction(self, image_path):
        """делает предсказание на одной картинке, расположенной по пути image_path
         для модели нейросети для классифкации картинок"""

        # отключаем расчет градиентов, мы только предсказываем
        with torch.no_grad():
            tensor = self.transform_image(image_path=image_path)
            outputs = self.model.forward(tensor.to(self.device))
            _, prediction = torch.max(outputs, 1)
            prediction = prediction.cpu().detach().numpy()

        return self.label_enc.inverse_transform(prediction)[0]

    def get_prediction_on_image_folder(self, images_folder_path):
        """делает предсказание для всех картинок, расположенных в папке images_folder_path,
        моделью нейросети для классифкации картинок, результаты хранит в переменных:
        self.prediction_images_values -  предсказание нейросети
        self.prediction_images_names - название картинки
         """
        # для предсказания на папке файлов
        self.prediction_images_values = []
        self.prediction_images_names = []

        # получаем пути к изображениям
        images_filenames = os.listdir(images_folder_path)

        # предсказываем моделью
        # отключаем расчет градиентов, мы только предсказываем
        with torch.no_grad():
            for elem in images_filenames:
                pred = self.get_prediction(images_folder_path + elem)
                pred = 0 if pred == 'health' else 1
                self.prediction_images_values.append(pred)
                self.prediction_images_names.append(elem)

    def get_csv_prediction_on_image_folder(self, output_csv_filename, csv_index_flag = False):
        """создает csv-файл с именем output_csv_filename с предсказаниями модели,
        если csv_index_flag == True в csv-файле колонка-индекс картинок.
        предсказания должны быть предварительно расчитаны методом 
        get_prediction_on_image_folder() !
        Нужно для сдачи модели менторам.
        """

        # задаем заголовок csv
        csv_columns = ['disease_flag','name']

        # создаем csv
        results_df = pd.DataFrame(list(zip(self.prediction_images_values, self.prediction_images_names)), columns=csv_columns)
        results_df.to_csv(output_csv_filename, index=csv_index_flag)

    def get_dct_prediction_on_image_folder(self):
        """создает python dict с предсказаниями модели,
        предсказания должны быть предварительно расчитаны методом 
        get_prediction_on_image_folder() !
        Нужно для веб-сервиса
        """

        dct = {}
        for i, k in zip(self.prediction_images_names, self.prediction_images_values):
            dct[i] = k

        return dct



    def create_dataloader_from_folder(self, path_to_images_folder):
        pass

    def predict_on_dataloader(self):
        pass
        # return predictions

    def get_csv_prediction_on_dataloader(self):
        pass
        # create csv - с предсказаниями

    def get_dict_prediction_on_dataloader(self):
        pass
        # create dict {./img/tree_1:1, ./img/tree_2: 0, ./img/tree_3: 0, ./img/tree_4: 1}

In [ ]:
# путь к сохраненной модели
path_model = '/content/gdrive/MyDrive/AgroCode_pre_snippets/model/ResNet34_DGL_AdamW_aug_oversampling_batchsize_16_full_model.pth'

# путь к сохраненному кодировщику классов
path_label_enc = '/content/gdrive/MyDrive/AgroCode_pre_snippets/model/label_encoder.pkl'

In [ ]:
# создадим экземпляр класса нашнй модели и передадим ей пути к сохраненной нейрости
# и кодировщику классов
web_dl_model = DLModelImageClassifier(path_model, path_label_enc)

Предсказание на одной картинке

In [ ]:
image_path = '/content/gdrive/MyDrive/AgroCode_pre_snippets/pics_example/00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335.JPG'

In [ ]:
# отработал на CPU - сессии Colaba
web_dl_model.device

device(type='cpu')

In [ ]:
%%time
# сделаем предсказание
web_dl_model.get_prediction(image_path)

CPU times: user 191 ms, sys: 16.1 ms, total: 207 ms
Wall time: 213 ms


'sick'

In [ ]:
# отработал на GPU - сессии Colaba
web_dl_model.device

device(type='cpu')

In [ ]:
%%time
# сделаем предсказание
web_dl_model.get_prediction(image_path)

CPU times: user 179 ms, sys: 763 µs, total: 179 ms
Wall time: 182 ms


'sick'

Предсказание на папке картинок:

In [ ]:
images_folder_path = '//content//dataset//health//'
output_csv_filename = 'predictions.csv'

In [ ]:
%%time
# сделаем предсказание на папке с картинками health 880
web_dl_model.get_prediction_on_image_folder(images_folder_path)

CPU times: user 3min 29s, sys: 3.68 s, total: 3min 33s
Wall time: 3min 33s


In [ ]:
# тут результаты предсказаний
web_dl_model.prediction_images_values

In [ ]:
# тут имена картинок
web_dl_model.prediction_images_names

In [ ]:
# получим csv-файл предсказаний
web_dl_model.get_csv_prediction_on_image_folder(output_csv_filename, csv_index_flag = False)

In [ ]:
# получим python dict предсказаний
web_dl_model.get_dct_prediction_on_image_folder()

{'Train_0.jpg': 0,
 'Train_1.jpg': 0,
 'Train_10.jpg': 0,
 'Train_100.jpg': 0,
 'Train_1000.jpg': 0,
 'Train_1001.jpg': 0,
 'Train_1005.jpg': 0,
 'Train_1006.jpg': 0,
 'Train_1007.jpg': 0,
 'Train_1008.jpg': 0,
 'Train_1010.jpg': 0,
 'Train_1014.jpg': 0,
 'Train_1015.jpg': 0,
 'Train_1017.jpg': 0,
 'Train_1018.jpg': 0,
 'Train_1019.jpg': 0,
 'Train_1022.jpg': 0,
 'Train_1024.jpg': 0,
 'Train_1026.jpg': 0,
 'Train_1028.jpg': 0,
 'Train_1034.jpg': 0,
 'Train_1037.jpg': 0,
 'Train_1038.jpg': 0,
 'Train_1039.jpg': 0,
 'Train_104.jpg': 0,
 'Train_1040.jpg': 0,
 'Train_1041.jpg': 0,
 'Train_1042.jpg': 0,
 'Train_1045.jpg': 0,
 'Train_1047.jpg': 0,
 'Train_1048.jpg': 0,
 'Train_1049.jpg': 0,
 'Train_1050.jpg': 0,
 'Train_1051.jpg': 0,
 'Train_1052.jpg': 0,
 'Train_1053.jpg': 0,
 'Train_1054.jpg': 0,
 'Train_1057.jpg': 0,
 'Train_1059.jpg': 0,
 'Train_106.jpg': 0,
 'Train_1061.jpg': 0,
 'Train_1062.jpg': 0,
 'Train_1065.jpg': 0,
 'Train_1066.jpg': 0,
 'Train_1067.jpg': 0,
 'Train_1069.jpg': 0,